In [2]:
# Data-handling libraries
import pandas as pd
import scipy.sparse

# Feature extraction/transformation libraries
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso, LassoCV
from sklearn.decomposition import PCA

# Modelling libraries
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

# Random seed
seed = 5777

# Import data

In [3]:
essays = {}
for i in range(1, 9):
    essays[i] = pd.read_csv(f"../data_private/essays_cleaned_set_{i}.csv")

# Select/extract features

In [4]:
target_set = essays[1]

X = target_set.drop(columns="score")
y = target_set["score"]

## Vectorize text

In [5]:
X_cvec = CountVectorizer(
        ngram_range = (1,2),
        min_df = 2, # ignore hapaxes
    ).fit_transform(X["essay"]);

X = scipy.sparse.csr_matrix(np.concatenate([
    X.drop(columns="essay").values,
    X_cvec.todense()
], axis=1))

## Scale all features

In [6]:
X = StandardScaler().fit_transform(X);

ValueError: Cannot center sparse matrices: pass `with_mean=False` instead. See docstring for motivation and alternatives.

## Select feature set

In [7]:
X = PCA(random_state = seed).fit_transform(X);

TypeError: PCA does not support sparse input. See TruncatedSVD for a possible alternative.

## Create train & test sets

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed);

# Fit models

In [13]:
gnb = GaussianNB()
gnb.fit(X_train.toarray(), y_train)

print("Train score:", gnb.score(X_train.toarray(), y_train))
print("Test score:", gnb.score(X_test.toarray(), y_test))

Train score: 1.0
Test score: 0.4304932735426009


In [14]:
# Set up a list of Lasso alphas to check.
l_alphas = np.logspace(-3, 0, 100)

# Cross-validate over our list of Lasso alphas.
lasso_cv = LassoCV(alphas=l_alphas, cv=5, max_iter=5000)

# Fit model using best ridge alpha!
lasso_cv.fit(X_train, y_train);

KeyboardInterrupt: 

In [ ]:
print(lasso_model.score(X_train, y_train))
print(lasso_model.score(X_test, y_test))

# Evaluate models